In [23]:
!pip install tiktoken

  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/16/05/5efbd91252ffb1301ea393d88ef736b33d41e75d4bcf0bd31d660050e400/tiktoken-0.6.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for regex>=2022.1.18 from https://files.pythonhosted.org/packages/81/8a/96a62ce98e8ff1b16db56fde3debc8a571f6b7ea42ee137eb0d995cdfa26/regex-2023.12.25-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 80.2 MB/s eta 0:00:00


In [8]:
import tiktoken

encoding = tiktoken.get_encoding("cl100k_base")

In [21]:
with open("input.txt", "r") as f:
    text = f.read()

In [7]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [8]:
encoded_text = encoding.encode(text)

In [11]:
len(encoded_text)

301829

In [12]:
len(text)

1115394

In [24]:
from src.dataset import TextData, NaiveEncoder

In [25]:
dataset = TextData("input.txt", batch_size=4, block_size=32)

In [9]:
x, y = dataset.get_batch()

In [10]:
x.shape

torch.Size([4, 32])

In [26]:
encoder = NaiveEncoder(text)

In [12]:
encoder.decode(x[0, :].numpy().tolist())

'uld think.\n\nSecond Servingman:\nS'

In [10]:
encoding.n_vocab

100277

In [3]:
import torch
from src.model import Decoder

In [4]:
model = Decoder(
    n_vocab=65,
    block_size=64,
    n_layer=3,
    n_embd=256,
    n_head=4,
    n_inner=1024,
)
model.load_state_dict(torch.load("models/2024-02-23_18-14-17/model_batch_size-32_block_size-64_encode_scheme-naive_lr-0.0006_n_embd-256_n_head-4_n_inner-1024_n_layer-3_iter-100000.pt"))

<All keys matched successfully>

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
model = model.to(device)

In [9]:
model.eval()
context = torch.zeros(1, 64, dtype=torch.long).to(device)

In [10]:
with torch.no_grad():
    logits = model(context)

/opt/conda/envs/pytorch/lib/python3.10/site-packages/torch/nn/modules/activation.py:1160: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /opt/conda/conda-bld/pytorch_1686274778240/work/aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._native_multi_head_attention(


In [11]:
logits.shape

torch.Size([1, 64, 65])

In [12]:
import torch.nn.functional as F

In [13]:
logits = logits[:, -1, :]
probs = F.softmax(logits, dim=-1)

In [15]:
probs.shape

torch.Size([1, 65])

In [16]:
idx_next = torch.multinomial(probs, num_samples=1)

In [18]:
idx_next.shape

torch.Size([1, 1])

In [19]:
context = torch.cat([context, idx_next], dim=-1)

In [20]:
context.shape

torch.Size([1, 65])

In [27]:
context.shape

torch.Size([1, 65])

In [29]:
context = context.to("cpu").tolist()

In [31]:
context

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  29]]

In [32]:
encoder.decode(context[0])

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nQ'